# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "../WeatherPy/output_data/cities.csv"
df = pd.read_csv(file, dtype="object", encoding="utf-8")
df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Tuktoyaktuk,20,CA,1578429029,66,69.44,-133.03,-32.8,12.75
1,1,Ushuaia,75,AR,1578428924,81,-54.81,-68.31,53.6,10.29
2,2,Busselton,26,AU,1578429032,55,-33.64,115.35,64.0,4.45
3,3,East London,100,ZA,1578428911,90,-33.02,27.91,73.2,16.6
4,4,Barrow,0,AR,1578429034,25,-38.31,-60.23,82.62,3.76


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
locations = df[["Lat", "Lng"]].astype(float)


In [5]:
# Convert humidity to float and store
humidity = df["Humidity"].astype(float)

In [6]:
# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
new_df = df

In [21]:
#Convert Columns to numerics
new_df["Max Temp"] = pd.to_numeric(df["Max Temp"])
new_df["Wind Speed"] = pd.to_numeric(df["Wind Speed"])
new_df["Cloudiness"] = pd.to_numeric(df["Cloudiness"])

In [22]:
max_temp = new_df[new_df['Max Temp'] > 85].index
min_temp = new_df[new_df['Max Temp']<65].index
wind = new_df[new_df['Wind Speed']>10].index
no_clouds = new_df[new_df['Cloudiness']>0].index  

new_df.drop(max_temp, inplace=True)
new_df.drop(min_temp, inplace=True)
new_df.drop(wind, inplace=True)
new_df.drop(no_clouds, inplace=True)

In [24]:
new_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
4,4,Barrow,0,AR,1578429034,25,-38.31,-60.23,82.62,3.76
38,38,Bredasdorp,0,ZA,1578428943,83,-34.53,20.04,71.01,2.24
50,50,Cape Town,0,ZA,1578428842,94,-33.93,18.42,73.99,8.05
84,84,Vaini,0,IN,1578428940,79,15.34,74.49,65.88,1.43
105,105,Hermanus,0,ZA,1578429104,89,-34.42,19.24,70.00,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
hotel_df = []
new_df['Hotel'] = ""

In [43]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "nearest hotel",
    "key": g_key
}

for index, row in new_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name = requests.get(base_url, params=params)
    
    # convert to json
    name = name.json()

    try:
        new_df.loc[index, "hotel"] = name["results"][0]["name"]

    except (KeyError, IndexError):
        continue
        


In [44]:
new_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel,hotel
4,4,Barrow,0,AR,1578429034,25,-38.31,-60.23,82.62,3.76,,NaN
38,38,Bredasdorp,0,ZA,1578428943,83,-34.53,20.04,71.01,2.24,,A Piece Of Life
50,50,Cape Town,0,ZA,1578428842,94,-33.93,18.42,73.99,8.05,,Atlanticview Cape Town Boutique Hotel
84,84,Vaini,0,IN,1578428940,79,15.34,74.49,65.88,1.43,,Dandeli Dreams Jungle resorts
105,105,Hermanus,0,ZA,1578429104,89,-34.42,19.24,70.00,1.01,,Harbour House Hotel


In [95]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in new_df.iterrows()]
locations = new_df[["Lat", "Lng"]].astype(float)

In [98]:
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content = hotel_info)

In [99]:
#create combined map
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))